In [1]:
import pandas as pd
import os
import math

from bokeh.io import output_notebook
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, CDSView, GroupFilter

from bokeh.palettes import Spectral3
from bokeh.transform import factor_cmap

#import matplotlib.pyplot as plt
output_notebook()


from pathlib import *

Loading BokehJS ...

Tasks

1) Perform a left join on the summary and events data on Case ID
2) Develop 
3) Create Visualizations for Court Appearances by (Exclude Preventitive Detention)
    - Day of the week
    - Seasons 
    
4) Create Bokeh Visualizations on group by objects
5)

In [2]:
def aggregate_data():
    base_path = Path.cwd()
    parsed_path = Path(base_path.parent.parent, 'fta')
    
    events_parsed_files = sorted(parsed_path.rglob("events_data*.csv"))
    df_events = pd.concat(pd.read_csv(f) for f in events_parsed_files)

    summary_parsed_files = sorted(parsed_path.rglob("summary_data*.csv"))
    df_summary = pd.concat(pd.read_csv(f) for f in summary_parsed_files)

    data = pd.merge(df_events, df_summary, on= 'case_id')
    return data




In [3]:
# data_2015 contains all aggregated data now just didn't want to change the variable name

data_2015 = aggregate_data()

C:\Users\TJ\AppData\Local\Continuum\anaconda3\envs\feat_tools\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\Users\TJ\AppData\Local\Continuum\anaconda3\envs\feat_tools\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [4]:
data_2015.columns

Index(['Date', 'DateTime', 'Docket Text', 'Image Avail.', 'Judge', 'Location',
       'Result', 'Type', 'case_id', 'case_title_x', 'party_x', 'action',
       'case_status', 'case_title_y', 'case_type', 'file_date', 'next_event',
       'party_y', 'party_aka', 'party_attny', 'party_type', 'status_date'],
      dtype='object')

In [5]:
data_2015 = data_2015[['DateTime', 'Location', 'Type', 'Result', 'Judge', 'case_id']].copy()

In [6]:
data_2015.shape

(336351, 6)

In [7]:
data_2015.case_id.nunique()

32239

In [8]:
# Include detention status as a column
detention = ['Preventive Detention', 'Defendant detained pending sentencing','Defendant Held in Other Jurisdiction']
data_2015['detention'] = data_2015['Result'].apply(lambda x: 1 if x in detention else 0 )

In [9]:
data_2015.head()

DateTime        Location                 Type  \
0  01/01/2007 11:00 AM  Courtroom C-10          Presentment   
1  01/18/2007 09:30 AM   Courtroom 221   Felony Arraignment   
2  01/18/2007 11:00 AM   Courtroom 201  Preliminary Hearing   
3  02/01/2007 09:30 AM   Courtroom 221       Status Hearing   
4  03/02/2007 09:30 AM   Courtroom 221       Status Hearing   

                                             Result Judge          case_id  \
0                                      Hearing Held   NaN  2007 CF2 000003   
1  Defn Arraigned Plead Not Guilty Jury Demand Made   NaN  2007 CF2 000003   
2                                           Vacated   NaN  2007 CF2 000003   
3     Status, Preliminary or Other Hearing Not Held   NaN  2007 CF2 000003   
4                                      Hearing Held   NaN  2007 CF2 000003   

   detention  
0          0  
1          0  
2          0  
3          0  
4          0

In [10]:
data_2015['DateTime'] = pd.to_datetime(data_2015['DateTime'])

In [11]:
# Identify all Preventitive Detention cases
case_exclusions = ['Preventive Detention', 'Defendant Held in Other Jurisdiction','Defendant detained pending sentencing']
preventitive_data = data_2015.loc[data_2015['Result'].isin (case_exclusions)]


In [12]:
# Non-unique cases with defendants remanded 
preventitive_data.shape

(7565, 7)

In [13]:
# Non-unique cases with defendants remanded 
preventitive_data.shape

(7565, 7)

In [14]:
case_id_exclude = preventitive_data['case_id'].unique()

In [15]:
# Number of unique cases to exclude in the data_2015
len(preventitive_data['case_id'].unique())

7257

In [16]:
# Non-unique cases with defendants remanded 
preventitive_data.shape

(7565, 7)

In [17]:
# Exclude all preventitive detention cases from data_2015 
data_2015_nonpreventitive = data_2015.loc[~data_2015['case_id'].isin(case_id_exclude)].copy()


data_2015_preventitive = data_2015.loc[data_2015['case_id'].isin(case_id_exclude)].copy()

In [18]:
# Number unique case_ids in the non_preventative population
data_2015_nonpreventitive['case_id'].nunique()

24982

In [19]:
data_2015_nonpreventitive['case_id'].nunique()

24982

In [20]:
# Number of unique case_idsin preventitive population
data_2015_preventitive['case_id'].nunique()

7257

In [21]:
data_2015_preventitive['case_id'].nunique()

7257

In [22]:
# Include Day of the week to FTA Data
data_2015_nonpreventitive['Weekday'] = data_2015_nonpreventitive['DateTime'].dt.day_name()

# Include feature for time of the day - AM/ PM
data_2015_nonpreventitive['Period'] = data_2015_nonpreventitive['DateTime'].dt.strftime('%p')

# Include feature for month - AM/ PM
# data_2015_nonpreventitive['Month'] = data_2015_nonpreventitive['DateTime'].dt.month_name()
data_2015_nonpreventitive['Month'] = data_2015_nonpreventitive['DateTime'].dt.strftime('%m')

# Include feature for year
data_2015_nonpreventitive['Year'] = data_2015_nonpreventitive['DateTime'].dt.strftime('%Y')

In [23]:
data_2015_nonpreventitive.head()

DateTime        Location                 Type  \
0 2007-01-01 11:00:00  Courtroom C-10          Presentment   
1 2007-01-18 09:30:00   Courtroom 221   Felony Arraignment   
2 2007-01-18 11:00:00   Courtroom 201  Preliminary Hearing   
3 2007-02-01 09:30:00   Courtroom 221       Status Hearing   
4 2007-03-02 09:30:00   Courtroom 221       Status Hearing   

                                             Result Judge          case_id  \
0                                      Hearing Held   NaN  2007 CF2 000003   
1  Defn Arraigned Plead Not Guilty Jury Demand Made   NaN  2007 CF2 000003   
2                                           Vacated   NaN  2007 CF2 000003   
3     Status, Preliminary or Other Hearing Not Held   NaN  2007 CF2 000003   
4                                      Hearing Held   NaN  2007 CF2 000003   

   detention   Weekday Period Month  Year  
0          0    Monday     AM    01  2007  
1          0  Thursday     AM    01  2007  
2          0  Thursday     AM    01  2007  
3          0  Thursday     AM    02  2007  
4          0    Friday     AM    03  2007

In [24]:
# Frequency count of FTA cases of the non preventitivve cases
len(data_2015_nonpreventitive[data_2015_nonpreventitive['Result'] == 'Defendant Failed to Appear'])

10867

In [25]:
# Number of unique case_ids in the FTA population
data_2015_nonpreventitive.loc[data_2015_nonpreventitive['Result'] == 'Defendant Failed to Appear']['case_id'].nunique()

6739

In [26]:
406/1370

0.29635036496350364

In [27]:
# Frequency count of FTA cases of the non preventitivve cases
len(data_2015_nonpreventitive[data_2015_nonpreventitive['Result'] == 'Defendant Failed to Appear'])

10867

In [28]:
# Number of unique case_ids in the FTA population
data_2015_nonpreventitive.loc[data_2015_nonpreventitive['Result'] == 'Defendant Failed to Appear']['case_id'].nunique()

6739

In [29]:
# Identify FTA Cases
# Includes multiple repeated failure to appear for each case
FTA_data = data_2015_nonpreventitive[data_2015_nonpreventitive['Result'] == 'Defendant Failed to Appear'].copy()

In [30]:
FTA_data.columns

Index(['DateTime', 'Location', 'Type', 'Result', 'Judge', 'case_id',
       'detention', 'Weekday', 'Period', 'Month', 'Year'],
      dtype='object')

In [31]:
FTA_data.head()

DateTime       Location                 Type  \
62 2007-02-28 08:30:00  Courtroom 217   Felony Arraignment   
64 2007-04-26 08:30:00  Courtroom 217   Felony Arraignment   
74 2007-01-22 11:00:00  Courtroom 201  Preliminary Hearing   
76 2007-04-23 09:00:00  Courtroom 212       Status Hearing   
88 2007-01-24 09:30:00  Courtroom 221   Felony Arraignment   

                        Result Judge          case_id  detention    Weekday  \
62  Defendant Failed to Appear   NaN  2007 CF2 000034          0  Wednesday   
64  Defendant Failed to Appear   NaN  2007 CF2 000034          0   Thursday   
74  Defendant Failed to Appear   NaN  2007 CF2 000035          0     Monday   
76  Defendant Failed to Appear   NaN  2007 CF2 000035          0     Monday   
88  Defendant Failed to Appear   NaN  2007 CF2 000045          0  Wednesday   

   Period Month  Year  
62     AM    02  2007  
64     AM    04  2007  
74     AM    01  2007  
76     AM    04  2007  
88     AM    01  2007

In [32]:
# Checking groupings

In [33]:
FTA_by_day = FTA_data.groupby('DateTime')['case_id'].count().reset_index()

In [34]:
FTA_by_day.columns

Index(['DateTime', 'case_id'], dtype='object')

In [35]:
FTA_by_day.head()

DateTime  case_id
0 2007-01-11 09:00:00        1
1 2007-01-22 09:00:00        1
2 2007-01-22 11:00:00        1
3 2007-01-24 09:30:00        1
4 2007-01-25 09:00:00        1

In [36]:
# Realized Initiated cases span across multiple years
#plotline(FTA_by_day, "FTA by day")

In [37]:
FTA_by_weekday = FTA_data.groupby('Weekday')['case_id'].count().reset_index()

In [38]:
FTA_by_weekday.rename(columns = {'case_id': 'FTA_count'}, inplace= True)
FTA_by_weekday

Weekday  FTA_count
0     Friday       2558
1     Monday       1783
2   Thursday       1975
3    Tuesday       2320
4  Wednesday       2231

In [39]:
Weekday_count = data_2015_nonpreventitive.groupby('Weekday')['case_id'].count().reset_index()

In [40]:
Weekday_count

Weekday  case_id
0     Friday    53515
1     Monday    45647
2   Saturday     5987
3     Sunday       18
4   Thursday    50310
5    Tuesday    49549
6  Wednesday    51546

In [41]:
data_2015_nonpreventitive['Weekday'].unique()

array(['Monday', 'Thursday', 'Friday', 'Wednesday', 'Tuesday', 'Saturday',
       'Sunday', nan], dtype=object)

In [42]:
Weekday_overall = pd.merge(FTA_by_weekday, Weekday_count, on= 'Weekday')

In [43]:
Weekday_overall.rename(columns = {'case_id': 'Overall_count'}, inplace= True)
Weekday_overall

Weekday  FTA_count  Overall_count
0     Friday       2558          53515
1     Monday       1783          45647
2   Thursday       1975          50310
3    Tuesday       2320          49549
4  Wednesday       2231          51546

In [44]:
Weekday_overall.rename(columns = {'case_id': 'Overall_count'}, inplace= True)
Weekday_overall

Weekday  FTA_count  Overall_count
0     Friday       2558          53515
1     Monday       1783          45647
2   Thursday       1975          50310
3    Tuesday       2320          49549
4  Wednesday       2231          51546

In [45]:
# Sort Dataframe

def sort_dataframe (dataframe, sort_column, sorter):
    """
    Sort column in dataframe in a specified order
    
    """
    sorterIndex = dict(zip(sorter, range(len(sorter))))
    dataframe['order_id'] = dataframe[sort_column]
    dataframe['order_id'] = dataframe['order_id'].map(sorterIndex)
    dataframe.sort_values('order_id', inplace= True)
    return dataframe
    

In [46]:
# Sort days of the week in this order
sorter = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

sort_dataframe(Weekday_overall, 'Weekday', sorter )

Weekday  FTA_count  Overall_count  order_id
1     Monday       1783          45647         0
3    Tuesday       2320          49549         1
4  Wednesday       2231          51546         2
2   Thursday       1975          50310         3
0     Friday       2558          53515         4

In [47]:
# Sort days of the week in this order
sorter = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

sort_dataframe(Weekday_overall, 'Weekday', sorter )

Weekday  FTA_count  Overall_count  order_id
1     Monday       1783          45647         0
3    Tuesday       2320          49549         1
4  Wednesday       2231          51546         2
2   Thursday       1975          50310         3
0     Friday       2558          53515         4

In [48]:
# Calculate ratio of FTAs
Weekday_overall['Ratio'] = Weekday_overall['FTA_count'] /Weekday_overall['Overall_count']

# Calculate FTAs and Non_FTAs counts
Weekday_overall['Non_FTA'] = Weekday_overall['Overall_count'] - Weekday_overall['FTA_count']

Weekday_overall

Weekday  FTA_count  Overall_count  order_id     Ratio  Non_FTA
1     Monday       1783          45647         0  0.039061    43864
3    Tuesday       2320          49549         1  0.046822    47229
4  Wednesday       2231          51546         2  0.043282    49315
2   Thursday       1975          50310         3  0.039257    48335
0     Friday       2558          53515         4  0.047800    50957

In [49]:
# Calculate ratio of FTAs
Weekday_overall['Ratio'] = Weekday_overall['FTA_count'] /Weekday_overall['Overall_count']

# Calculate FTAs and Non_FTAs counts
Weekday_overall['Non_FTA'] = Weekday_overall['Overall_count'] - Weekday_overall['FTA_count']

Weekday_overall

Weekday  FTA_count  Overall_count  order_id     Ratio  Non_FTA
1     Monday       1783          45647         0  0.039061    43864
3    Tuesday       2320          49549         1  0.046822    47229
4  Wednesday       2231          51546         2  0.043282    49315
2   Thursday       1975          50310         3  0.039257    48335
0     Friday       2558          53515         4  0.047800    50957

In [50]:
# Creating Functions to plot graphs
from bokeh.core.properties import value
def make_barplot(dataframe, x_range, value, tool_tips, title, x_axis, y_axis):
    """
    Plots Bar chart
    """
    
    src = ColumnDataSource(dataframe)
    tooltips = tool_tips

    # define plot figure
    p = figure(x_range= dataframe[x_range], 
               plot_height=600,
               plot_width = 900,
               title=title,
               x_axis_label = x_axis,
               y_axis_label = y_axis,
               tooltips = tooltips, 
               toolbar_location=None
              )
    p.vbar(x=x_range,
           top=value,
           width=0.5,
           color = 'indianred',
           source = src,
           bottom=0,)
        
    #p.legend.location = "top_left"
    show(p)
    
def make_stackedplot(dataframe, x_range, tool_tips, title, x_axis, y_axis, stackers):
    """
    Plots stacked bars
    """
    
    src = ColumnDataSource(dataframe)
    tooltips = tool_tips

    # define plot figure
    p = figure(x_range= dataframe[x_range], 
               plot_height=600,
               plot_width = 900,
               title=title,
               x_axis_label = x_axis,
               
               y_axis_label = y_axis,
               tooltips = tooltips, 
               toolbar_location=None)

    p.vbar_stack(stackers, 
                 x = x_range, 
                 width=0.5, 
                 color = ['indianred', 'green'],
                 legend = [value(x) for x in stackers],
                 source = src)
        
    p.xaxis.major_label_orientation = math.pi/3
    p.legend.orientation = "horizontal"
    p.legend.location = "top_left"
    
#     #legend = {'Non_FTA': 'indianred'}, {'FTA_count':'green'}#, location=(0, 30))
#     p.add_layout(legend, 'right')
    
    show(p)

In [51]:
# Ratio of FTA's to Weekdays
Weekday_overall

Weekday  FTA_count  Overall_count  order_id     Ratio  Non_FTA
1     Monday       1783          45647         0  0.039061    43864
3    Tuesday       2320          49549         1  0.046822    47229
4  Wednesday       2231          51546         2  0.043282    49315
2   Thursday       1975          50310         3  0.039257    48335
0     Friday       2558          53515         4  0.047800    50957

In [52]:
# Ratio of FTA's to Weekdays
Weekday_overall

Weekday  FTA_count  Overall_count  order_id     Ratio  Non_FTA
1     Monday       1783          45647         0  0.039061    43864
3    Tuesday       2320          49549         1  0.046822    47229
4  Wednesday       2231          51546         2  0.043282    49315
2   Thursday       1975          50310         3  0.039257    48335
0     Friday       2558          53515         4  0.047800    50957

In [53]:
tool_tips = [('FTA', '@FTA_count')]
title = 'Ratio of FTA cases'

# Call bar plot function
make_barplot(Weekday_overall, 'Weekday','Ratio', tool_tips, title, 'Weekday', 'Count')

In [54]:
Weekday_overall

Weekday  FTA_count  Overall_count  order_id     Ratio  Non_FTA
1     Monday       1783          45647         0  0.039061    43864
3    Tuesday       2320          49549         1  0.046822    47229
4  Wednesday       2231          51546         2  0.043282    49315
2   Thursday       1975          50310         3  0.039257    48335
0     Friday       2558          53515         4  0.047800    50957

In [55]:
tool_tips = [('FTA', '@FTA_count'),('Non_FTA', '@Non_FTA')]
title = "Ratio of Weekday FTAs from 2007 - 2019"
stackers = ['Non_FTA', 'FTA_count']

# call stacked plot function
make_stackedplot(Weekday_overall,'Weekday', tool_tips, title,'Weekday', 'Count', stackers)

In [56]:
# Ratio of FTAs to Daytime

In [57]:
FTA_by_weekday_time = FTA_data.groupby(['Weekday', 'Period'])['case_id'].count().reset_index()

In [58]:
FTA_by_weekday_time

Weekday Period  case_id
0     Friday     AM     1979
1     Friday     PM      579
2     Monday     AM     1667
3     Monday     PM      116
4   Thursday     AM     1861
5   Thursday     PM      114
6    Tuesday     AM     1905
7    Tuesday     PM      415
8  Wednesday     AM     2034
9  Wednesday     PM      197

In [59]:
# Concatenate Week_day Period

FTA_by_weekday_time['Weekday-Period'] = FTA_by_weekday_time['Weekday']+ '-' + FTA_by_weekday_time['Period']

In [60]:
FTA_by_weekday_time.head()

Weekday Period  case_id Weekday-Period
0    Friday     AM     1979      Friday-AM
1    Friday     PM      579      Friday-PM
2    Monday     AM     1667      Monday-AM
3    Monday     PM      116      Monday-PM
4  Thursday     AM     1861    Thursday-AM

In [61]:
# Check this against the total statistics of case schedule times (AM/PM)

In [62]:
data_2015_nonpreventitive.columns

Index(['DateTime', 'Location', 'Type', 'Result', 'Judge', 'case_id',
       'detention', 'Weekday', 'Period', 'Month', 'Year'],
      dtype='object')

In [63]:
data_2015_nonpreventitive['Weekday-Period'] = data_2015_nonpreventitive['Weekday']+ '-' + data_2015_nonpreventitive['Period']

In [64]:
data_2015_nonpreventitive_overall = data_2015_nonpreventitive.groupby('Weekday-Period')['case_id'].count().reset_index()

In [65]:
data_2015_nonpreventitive_overall.head()

Weekday-Period  case_id
0      Friday-AM    42322
1      Friday-PM    11193
2      Monday-AM    38586
3      Monday-PM     7061
4    Saturday-AM     3372

In [66]:
data_2015_Weekday_Period = pd.merge(FTA_by_weekday_time, data_2015_nonpreventitive_overall, on= 'Weekday-Period')

In [67]:
data_2015_Weekday_Period

Weekday Period  case_id_x Weekday-Period  case_id_y
0     Friday     AM       1979      Friday-AM      42322
1     Friday     PM        579      Friday-PM      11193
2     Monday     AM       1667      Monday-AM      38586
3     Monday     PM        116      Monday-PM       7061
4   Thursday     AM       1861    Thursday-AM      43050
5   Thursday     PM        114    Thursday-PM       7260
6    Tuesday     AM       1905     Tuesday-AM      43708
7    Tuesday     PM        415     Tuesday-PM       5841
8  Wednesday     AM       2034   Wednesday-AM      44714
9  Wednesday     PM        197   Wednesday-PM       6832

In [68]:
# Draw stacked graph on period also attempt to order it
data_2015_Weekday_Period.rename(columns = {'case_id_x': 'FTA_count', 'case_id_y': 'Overall_count'}, inplace= True)

In [69]:
data_2015_Weekday_Period['Weekday-Period'].unique()

array(['Friday-AM', 'Friday-PM', 'Monday-AM', 'Monday-PM', 'Thursday-AM',
       'Thursday-PM', 'Tuesday-AM', 'Tuesday-PM', 'Wednesday-AM',
       'Wednesday-PM'], dtype=object)

In [70]:
sorter = ['Monday-AM', 'Monday-PM', 'Tuesday-AM', 'Tuesday-PM', 'Wednesday-AM',
       'Wednesday-PM', 'Thursday-AM','Thursday-PM', 'Friday-AM', 'Friday-PM']

sort_dataframe (data_2015_Weekday_Period, 'Weekday-Period', sorter)

Weekday Period  FTA_count Weekday-Period  Overall_count  order_id
2     Monday     AM       1667      Monday-AM          38586         0
3     Monday     PM        116      Monday-PM           7061         1
6    Tuesday     AM       1905     Tuesday-AM          43708         2
7    Tuesday     PM        415     Tuesday-PM           5841         3
8  Wednesday     AM       2034   Wednesday-AM          44714         4
9  Wednesday     PM        197   Wednesday-PM           6832         5
4   Thursday     AM       1861    Thursday-AM          43050         6
5   Thursday     PM        114    Thursday-PM           7260         7
0     Friday     AM       1979      Friday-AM          42322         8
1     Friday     PM        579      Friday-PM          11193         9

In [71]:
data_2015_Weekday_Period['Non_FTA'] = data_2015_Weekday_Period['Overall_count'] - data_2015_Weekday_Period['FTA_count']

data_2015_Weekday_Period['Ratio'] = (data_2015_Weekday_Period['FTA_count'] / data_2015_Weekday_Period['Overall_count']) * 100

data_2015_Weekday_Period

Weekday Period  FTA_count Weekday-Period  Overall_count  order_id  \
2     Monday     AM       1667      Monday-AM          38586         0   
3     Monday     PM        116      Monday-PM           7061         1   
6    Tuesday     AM       1905     Tuesday-AM          43708         2   
7    Tuesday     PM        415     Tuesday-PM           5841         3   
8  Wednesday     AM       2034   Wednesday-AM          44714         4   
9  Wednesday     PM        197   Wednesday-PM           6832         5   
4   Thursday     AM       1861    Thursday-AM          43050         6   
5   Thursday     PM        114    Thursday-PM           7260         7   
0     Friday     AM       1979      Friday-AM          42322         8   
1     Friday     PM        579      Friday-PM          11193         9   

   Non_FTA     Ratio  
2    36919  4.320220  
3     6945  1.642827  
6    41803  4.358470  
7     5426  7.104948  
8    42680  4.548911  
9     6635  2.883489  
4    41189  4.322880  
5     7146  1.570248  
0    40343  4.676055  
1    10614  5.172876

In [72]:
tool_tips = [('FTA', '@Ratio')]
title = 'Ratio of FTA cases from 2007-2019 AM-PM'

# Call bar plot function
make_barplot(data_2015_Weekday_Period, 'Weekday-Period','Ratio', tool_tips, title, 'Weekday', 'FTA Percentage (%)')

In [73]:
# call stacked plot function
tool_tips = [('FTA', '@FTA_count'),('Non_FTA', '@Non_FTA')]
title = "Ratio of Weekday FTAs from 2007-2019 based on AM-PM"
stackers = ['Non_FTA', 'FTA_count']
make_stackedplot(data_2015_Weekday_Period,'Weekday-Period', tool_tips, title,'Weekday', 'Count', stackers)

#make_stackedplot(Weekday_overall,'Weekday', tool_tips, title,'Weekday', 'Count', stackers)

In [74]:
FTA_data.head()

DateTime       Location                 Type  \
62 2007-02-28 08:30:00  Courtroom 217   Felony Arraignment   
64 2007-04-26 08:30:00  Courtroom 217   Felony Arraignment   
74 2007-01-22 11:00:00  Courtroom 201  Preliminary Hearing   
76 2007-04-23 09:00:00  Courtroom 212       Status Hearing   
88 2007-01-24 09:30:00  Courtroom 221   Felony Arraignment   

                        Result Judge          case_id  detention    Weekday  \
62  Defendant Failed to Appear   NaN  2007 CF2 000034          0  Wednesday   
64  Defendant Failed to Appear   NaN  2007 CF2 000034          0   Thursday   
74  Defendant Failed to Appear   NaN  2007 CF2 000035          0     Monday   
76  Defendant Failed to Appear   NaN  2007 CF2 000035          0     Monday   
88  Defendant Failed to Appear   NaN  2007 CF2 000045          0  Wednesday   

   Period Month  Year  
62     AM    02  2007  
64     AM    04  2007  
74     AM    01  2007  
76     AM    04  2007  
88     AM    01  2007

In [75]:
# Frequency of FTAs initiated in 2015 over the years

In [76]:
FTA_by_month_year = FTA_data.groupby(['Year', 'Month'])['case_id'].count().reset_index()

In [77]:
# creating ColumnDataSource
breakdown = ColumnDataSource(FTA_by_month_year)



# creating views for each year in the 2015 data
view_2015 = CDSView(source = breakdown, 
                    filters = [GroupFilter(column_name = 'Year', group = '2015')])
view_2016 = CDSView(source = breakdown,
                       filters = [GroupFilter(column_name = 'Year', group = '2016')])
view_2017 = CDSView(source = breakdown,
                       filters = [GroupFilter(column_name = 'Year', group = '2017')])
view_2018 = CDSView(source = breakdown,
                       filters = [GroupFilter(column_name = 'Year', group = '2018')])
view_2019 = CDSView(source = breakdown,
                       filters = [GroupFilter(column_name = 'Year', group = '2019')])

tool_tips = tool_tips = [('FTA', '@case_id'),('Month', '@Month'),('Year', '@Year')]

# [('FTA', '@FTA_count'),('Non_FTA', '@Non_FTA')]

fig = figure(#x_range= 'Month',
             #x_axis_type = 'datetime',
             plot_height=600, plot_width=800,
             title='FTA for 2015 Initiated Cases',
             x_axis_label='Month', 
             y_axis_label='FTA Count',
             tooltips = tool_tips,
             toolbar_location=None)

fig.line('Month', 'case_id',
                 source = breakdown, view = view_2015, color = '#CE1141', legend = '2015 FTAs')
fig.line('Month', 'case_id',
                 source = breakdown, view = view_2016, color = '#808080', legend = '2016 FTAs')
fig.line('Month', 'case_id',
                 source = breakdown, view = view_2017, color = '#ffd500', legend = '2017 FTAs')
fig.line('Month', 'case_id',
                 source = breakdown, view = view_2018, color = '#008cff', legend = '2018 FTAs')
fig.line('Month', 'case_id',
                 source = breakdown, view = view_2019, color = '#41ce11', legend = '2019 FTAs')


show(fig)



In [78]:
#FTA_data['Type'].unique()

In [79]:
# Frequency Count for FTAs by Type
FTA_type = FTA_data.groupby('Type')['case_id'].count().reset_index()

In [80]:
# Frequency count of all types
overall_FTA_type = data_2015_nonpreventitive.groupby('Type')['case_id'].count().reset_index()

In [81]:
FTA_type = pd.merge(FTA_type, overall_FTA_type, on= 'Type')

In [82]:
FTA_type.rename(columns = {'case_id_x': 'FTA_count', 'case_id_y': 'Overall_count'}, inplace= True)

FTA_type.columns

FTA_type['Ratio'] = (FTA_type['FTA_count'] /FTA_type['Overall_count']) * 100

FTA_type['Non_FTA'] = FTA_type['Overall_count'] - FTA_type['FTA_count']

FTA_type.sort_values(by= 'FTA_count', ascending = False)
#df.sort_values(by='col1', ascending=False)

Type  FTA_count  \
39                      Probation Show Cause Hearing       2118   
43                                    Status Hearing       1793   
14                                Felony Arraignment       1615   
41                                  Sanction Hearing       1329   
34                               Preliminary Hearing       1172   
36                       Pretrial Show Cause Hearing        652   
16                        Full Report and Sentencing        306   
15                          Felony Status Conference        258   
18                                        Jury Trial        225   
33                     Post Trial Show Cause Hearing        175   
42                                        Sentencing        173   
37                          Probation Review Hearing        134   
5                                        Arraignment        130   
32                   Post Disposition Status Hearing        108   
40                                    Review Hearing        107   
21                      Mental Health Status Hearing         93   
22                        Mental Observation Hearing         71   
8                       Bench Warrant Return Hearing         68   
44                           Trial Readiness Hearing         47   
19                   Mental Health Admission Hearing         38   
4                Amended Sentencing Agreement Review         35   
26                                    Non-Jury Trial         34   
2                                 Admissions Hearing         33   
23                Misdemeanor Initial Status Hearing         21   
9                                Bond Review Hearing         15   
24                                    Motion Hearing         14   
12                        Deferred Sentencing Review         13   
7                           Ascertainment of Counsel         13   
11                                 Challenge Hearing         12   
13                          Diversion Status Hearing         12   
38                      Probation Revocation Hearing         10   
35                                       Presentment          7   
3                           After Sentencing Hearing          5   
0                                     23-1329 Review          4   
28  Post Disposition MH Probation Show Cause Hearing          4   
27                     Partial Report for Sentencing          4   
10                            Case Carried for Trial          4   
6                             Arraignment - Citation          3   
30  Post Disposition Mental Health Admission Hearing          2   
29       Post Disposition MH Probation Status Review          2   
20            Mental Health Diversion Status Hearing          2   
45                                     Trial Resumed          2   
31                   Post Disposition Motion Hearing          1   
25                                   Motions Resumed          1   
1                        24 Hour Gerstein Perfection          1   
17                      Halfway House Remand Hearing          1   

    Overall_count      Ratio  Non_FTA  
39          16938  12.504428    14820  
43          53612   3.344401    51819  
14          20563   7.853912    18948  
41           7769  17.106449     6440  
34          31694   3.697861    30522  
36           3471  18.784212     2819  
16          12002   2.549575    11696  
15          13649   1.890248    13391  
18          12539   1.794401    12314  
33            974  17.967146      799  
42           7298   2.370512     7125  
37           1915   6.997389     1781  
5            2302   5.647263     2172  
32            593  18.212479      485  
40           4535   2.359427     4428  
21           1662   5.595668     1569  
22           3539   2.006216     3468  
8           13051   0.521033    12983  
44           2299   2.044367     2252  
19            762   4.986877      724  
4             945   3.703704      910  
26           1328 

In [83]:
# Plot of top frequency FTA Counts 
tool_tips = [('FTA', '@FTA_count'),('FTA_Percentage', '@Ratio%'), ('Non_FTA', '@Non_FTA')]
title = "Top FTA Frequency Count Types for Case Stages "
stackers = ['Non_FTA', 'FTA_count']

# make_stackedplot(dataframe, x_range, tool_tips, title, x_axis, y_axis, stackers)
make_stackedplot(FTA_type[FTA_type.FTA_count > 7],'Type', tool_tips, title,'Case Type', 'Count', stackers)

In [84]:
# Check Judge's Statistics for FTA
data_2015['Judge'].unique()
data_2015.columns

Index(['DateTime', 'Location', 'Type', 'Result', 'Judge', 'case_id',
       'detention'],
      dtype='object')

In [85]:
# Missing Judge data can't help establish rate at which defendants are FTA for certain judges
# Only FTA data
FTA_Judge = FTA_data.groupby('Judge')['case_id'].count().reset_index()

# All 2015 Data
FTA_Judge_2015 = data_2015.groupby('Judge')['case_id'].count().reset_index()

Judges_FTA = pd.merge(FTA_Judge, FTA_Judge_2015, on= 'Judge')



In [86]:
Judges_FTA.rename(columns = {'case_id_x': 'FTA_count', 'case_id_y': 'Overall_count'}, inplace= True)

FTA_type.columns

Judges_FTA['Ratio'] = (Judges_FTA['FTA_count'] /Judges_FTA['Overall_count']) * 100

Judges_FTA['Non_FTA'] = Judges_FTA['Overall_count'] - Judges_FTA['FTA_count']

Judges_FTA.sort_values(by= 'FTA_count', ascending = False)
#df.sort_values(by='col1', ascending=False)

Judge  FTA_count  Overall_count     Ratio  Non_FTA
21     KNOWLES, KIMBERLEY        689          19936  3.456059    19247
38     TRAFFORD, SHERRY M        563          13381  4.207458    12818
13    DORSEY, FREDERICK D        497           7762  6.402989     7265
11        DEMEO, MARISA J        483          16552  2.918076    16069
2           ARTHUR, ERROL        448          12883  3.477451    12435
10        DAYSON, DANYA A        413          10139  4.073380     9726
15     HARRIS EPPS, DIANA        394          11889  3.313988    11495
14     EPSTEIN, ANTHONY C        345           9390  3.674121     9045
31      PASICHOW, HEIDI M        337           9215  3.657081     8878
37        STAPLES, SEAN C        321          11101  2.891631    10780
33     RAFFINAN, MARIBETH        312           7319  4.262877     7007
6      BESHOURI, JOSEPH E        171           5433  3.147432     5262
5          BERK, STEVEN N        160           4456  3.590664     4296
16       HOLEMAN, BRIAN F        154           4678  3.292005     4524
34      SALERNO, ROBERT A        138           4870  2.833676     4732
26         NASH, STUART G        128           5003  2.558465     4875
12    DI TORO, JENNIFER A        127           2462  5.158408     2335
18   IRVING, JR, ALFRED S        114           3517  3.241399     3403
29        PAN, FLORENCE Y        106           3491  3.036379     3385
30         PARKER, LORI E         92           3709  2.480453     3617
7          BRANDT, RAINEY         75           4275  1.754386     4200
41       WILLIAMS, YVONNE         73           2185  3.340961     2112
19     KING, III, RUFUS G         72           2359  3.052141     2287
24         LEVIE, RICHARD         62           1847  3.356795     1785
3        BEAUDIN, BRUCE D         53           1664  3.185096     1611
1      ALPRIN, GEOFFREY M         33            873  3.780069      840
35        SMITH, JUDITH A         27           1301  2.075327     1274
28         OKUN, ROBERT D         24           1146  2.094241     1122
42       WYNN, PATRICIA A         16            378  4.232804      362
0     ABRECHT, MARY ELLEN         15            467  3.211991      452
9         DALTON, CAROL A         12            206  5.825243      194
27         NOTI, ADRIENNE         11            536  2.052239      525
40  WIEDMANN, KATHERINE M          7             83  8.433735       76
25    MATINI, SHANA FROST          5            240  2.083333      235
17       IRVING, ALFRED S          5             71  7.042254       66
8        CORDERO, LAURA A          5            190  2.631579      185
23        LEPLEY, DIANE S          2             69  2.898551       67
32    PITTMAN, JONATHAN H          1             19  5.263158       18
22    KRAUTHAMER, PETER A          1            256  0.390625      255
20          KING, RUFUS G          1             67  1.492537       66
36      SOLTYS, DARLENE M          1             45  2.222222       44
39      WELLNER, STEVEN M          1             69  1.449275       68
4         BECKER, JULIE H          1             34  2.941176       33

In [87]:
# Plot of top frequency FTA Counts 
tool_tips = [('FTA', '@FTA_count'),('FTA_Percentage', '@Ratio%'), ('Non_FTA', '@Non_FTA')]
title = "Judges with most FTAs "
stackers = ['Non_FTA', 'FTA_count']

# make_stackedplot(dataframe, x_range, tool_tips, title, x_axis, y_axis, stackers)
make_stackedplot(Judges_FTA[Judges_FTA.FTA_count > 8],'Judge', tool_tips, title,'Judge Name', 'Count', stackers)

# What percentage of people held in detention FTA?

In [88]:
data_2015.head()

DateTime        Location                 Type  \
0 2007-01-01 11:00:00  Courtroom C-10          Presentment   
1 2007-01-18 09:30:00   Courtroom 221   Felony Arraignment   
2 2007-01-18 11:00:00   Courtroom 201  Preliminary Hearing   
3 2007-02-01 09:30:00   Courtroom 221       Status Hearing   
4 2007-03-02 09:30:00   Courtroom 221       Status Hearing   

                                             Result Judge          case_id  \
0                                      Hearing Held   NaN  2007 CF2 000003   
1  Defn Arraigned Plead Not Guilty Jury Demand Made   NaN  2007 CF2 000003   
2                                           Vacated   NaN  2007 CF2 000003   
3     Status, Preliminary or Other Hearing Not Held   NaN  2007 CF2 000003   
4                                      Hearing Held   NaN  2007 CF2 000003   

   detention  
0          0  
1          0  
2          0  
3          0  
4          0

In [89]:
focus_data = data_2015[data_2015['Result'] == 'Defendant Failed to Appear']

In [90]:
focus_data.shape

(12664, 7)

In [91]:
sum(focus_data.detention) #0 There was no case where the defendant failed to appear while in detention
focus_data[focus_data['detention'] == 1]

Empty DataFrame
Columns: [DateTime, Location, Type, Result, Judge, case_id, detention]
Index: []

In [92]:
# Count of FTAs for individual cases - circle plots
# Check Weekend Activity